In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import  RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectPercentile, GenericUnivariateSelect
from sklearn.feature_selection import RFE, RFECV
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import f_classif, chi2
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier, LassoCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import OrdinalEncoder

%matplotlib inline

In [5]:
salaryClassification = pd.read_csv('data/training.csv', sep = ';')

#treat missing values
salaryClassification = salaryClassification.replace(' ?', np.NaN)
salaryClassification[' workclass'] = salaryClassification[' workclass'].replace(np.NaN, 'Unknown')
salaryClassification[' occupation'] = salaryClassification[' occupation'].replace(np.NaN, 'Other')
salaryClassification[' native-country'] = salaryClassification[' native-country'].replace(np.NaN, 'Other')


#convert 'workclass' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["workclass"] = oe.fit_transform(salaryClassification[[" workclass"]]).astype(int)
salaryClassification = salaryClassification.drop(' workclass', 1)

#fix column names
salaryClassification['fnlwgt'] = salaryClassification[' fnlwgt']
salaryClassification = salaryClassification.drop(' fnlwgt',1)

#convert 'education' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["education"] = oe.fit_transform(salaryClassification[[" education"]]).astype(int)
salaryClassification = salaryClassification.drop(' education', 1)

#fix column names
salaryClassification['education-num'] = salaryClassification[' education-num']
salaryClassification = salaryClassification.drop(' education-num',1)

#convert 'education' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["marital-status"] = oe.fit_transform(salaryClassification[[" marital-status"]]).astype(int)
salaryClassification = salaryClassification.drop(' marital-status', 1)

#convert 'occupation'n' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["occupation"] = oe.fit_transform(salaryClassification[[" occupation"]]).astype(int)
salaryClassification = salaryClassification.drop(' occupation', 1)

#convert 'relationship'n' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["relationship"] = oe.fit_transform(salaryClassification[[" relationship"]]).astype(int)
salaryClassification = salaryClassification.drop(' relationship', 1)

#convert 'race'n' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["race"] = oe.fit_transform(salaryClassification[[" race"]]).astype(int)
salaryClassification = salaryClassification.drop(' race', 1)

#convert 'sex'n' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["sex"] = oe.fit_transform(salaryClassification[[" sex"]]).astype(int)
salaryClassification = salaryClassification.drop(' sex', 1)

#join 2 columns(capital-gain and  capital-loss) in one
salaryClassification['capital-diff'] = salaryClassification[' capital-gain'] - salaryClassification[' capital-loss']
salaryClassification = salaryClassification.drop(' capital-gain', 1)
salaryClassification = salaryClassification.drop(' capital-loss', 1)

#fix column names
salaryClassification['hours-per-week'] = salaryClassification[' hours-per-week']
salaryClassification = salaryClassification.drop(' hours-per-week',1)


#convert 'native-country' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["native-county"] = oe.fit_transform(salaryClassification[[" native-country"]]).astype(int)
salaryClassification = salaryClassification.drop(' native-country', 1)


#convert salary-classification' from categorical to numeric
salaryClassification[' salary-classification'] = [x.replace(' <=50K', '0') for x in salaryClassification[' salary-classification']]
salaryClassification[' salary-classification'] = [x.replace(' >50K', '1') for x in salaryClassification[' salary-classification']]
salaryClassification[' salary-classification'] = salaryClassification[' salary-classification'].astype(int)

salaryClassification['salary-classification'] = salaryClassification[' salary-classification']
salaryClassification = salaryClassification.drop(' salary-classification',1)



## teste 

testData = pd.read_csv('data/test.csv', sep = ';')

#treat missing values
testData = testData.replace(' ?', np.NaN)
testData[' workclass'] = testData[' workclass'].replace(np.NaN, 'Unknown')
testData[' occupation'] = testData[' occupation'].replace(np.NaN, 'Other')
testData[' native-country'] = testData[' native-country'].replace(np.NaN, 'Other')


#convert 'workclass' from categorical to numeric
oe = OrdinalEncoder()
testData["workclass"] = oe.fit_transform(testData[[" workclass"]]).astype(int)
testData = testData.drop(' workclass', 1)

#fix column name
testData['fnlwgt'] = testData[' fnlwgt']
testData = testData.drop(' fnlwgt',1)

#convert 'education' from categorical to numeric
oe = OrdinalEncoder()
testData["education"] = oe.fit_transform(testData[[" education"]]).astype(int)
testData = testData.drop(' education', 1)

#fix column name
testData['education-num'] = testData[' education-num']
testData = testData.drop(' education-num',1)

#convert 'education' from categorical to numeric
oe = OrdinalEncoder()
testData["marital-status"] = oe.fit_transform(testData[[" marital-status"]]).astype(int)
testData = testData.drop(' marital-status', 1)

#convert 'occupation'n' from categorical to numeric
oe = OrdinalEncoder()
testData["occupation"] = oe.fit_transform(testData[[" occupation"]]).astype(int)
testData = testData.drop(' occupation', 1)

#convert 'relationship'n' from categorical to numeric
oe = OrdinalEncoder()
testData["relationship"] = oe.fit_transform(testData[[" relationship"]]).astype(int)
testData = testData.drop(' relationship', 1)

#convert 'race'n' from categorical to numeric
oe = OrdinalEncoder()
testData["race"] = oe.fit_transform(testData[[" race"]]).astype(int)
testData = testData.drop(' race', 1)

#convert 'sex'n' from categorical to numeric
oe = OrdinalEncoder()
testData["sex"] = oe.fit_transform(testData[[" sex"]]).astype(int)
testData = testData.drop(' sex', 1)

#join 2 columns(capital-gain and  capital-loss) in one
testData['capital-diff'] = testData[' capital-gain'] - testData[' capital-loss']
testData = testData.drop(' capital-gain', 1)
testData = testData.drop(' capital-loss', 1)

#fix column name
testData['hours-per-week'] = testData[' hours-per-week']
testData = testData.drop(' hours-per-week',1)


#convert 'native-country' from categorical to numeric
oe = OrdinalEncoder()
testData["native-county"] = oe.fit_transform(testData[[" native-country"]]).astype(int)
testData = testData.drop(' native-country', 1)


#convert salary-classification' from categorical to numeric
testData[' salary-classification'] = [x.replace(' <=50K', '0') for x in testData[' salary-classification']]
testData[' salary-classification'] = [x.replace(' >50K', '1') for x in testData[' salary-classification']]
testData[' salary-classification'] = testData[' salary-classification'].astype(int)

testData['salary-classification'] = testData[' salary-classification']
testData = testData.drop(' salary-classification',1)